### From Load and Reprocessing

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
%run 'Load_and_Preprocessing.ipynb'

### Merge dataframes for analytics
df_orders: orders with tip <br>
df_order_products: ordered products with informations of the order, department and aisle.

In [ ]:
df_order_products = df_order_products
df_orders = df_orders

In [ ]:
df_orders['tip'] = df_orders['tip']*1
df_tip_to_merge = df_orders[['user_id', 'order_number', 'order_hour_of_day', 'days_since_prior_order', 'tip']]

In [ ]:

# 1.1
df_orders['previous_order_number'] = df_orders.apply(lambda x: x['order_number'] - 1, axis=1)
df_tip = df_orders[df_orders['previous_order_number'] != 0]
df_tip_analysis_1 =  pd.merge(left=df_tip, right=df_tip_to_merge, left_on=['user_id', 'previous_order_number'], right_on=['user_id', 'order_number'], how='inner', suffixes=['_next', '_previous'])
df_tip_analysis_1 = df_tip_analysis_1[['user_id', 'order_number_next', 'tip_next', 'order_number_previous', 'tip_previous']] 


In [ ]:
path=['tip_previous', 'tip_next']
px.parallel_categories(df_tip_analysis_1, dimensions=path, color='tip_next', title='Tip if tipped befored')

In [ ]:
print(f'Haben Trinkgeld bei der vorherigen Bestellung gegeben: {round(df_tip_analysis_1['tip_previous'].mean()*100, 3)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie vorher Trinkgeld gegeben haben:\
 {round(df_tip_analysis_1['tip_next'][df_tip_analysis_1['tip_previous']==1].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie vorher kein Trinkgeld gegeben haben:\
 {round(df_tip_analysis_1['tip_next'][df_tip_analysis_1['tip_previous']==0].mean()*100, 1)}%')


In [ ]:
# 1.2

df_orders['previous_order_number'] = df_orders.apply(lambda x: x['order_number'] - 1, axis=1)
df_orders['before_previous_order_number'] = df_orders.apply(lambda x: x['order_number'] - 2, axis=1)

df_tip = df_orders[df_orders['previous_order_number'] > 0]
df_tip = df_orders[df_orders['before_previous_order_number'] > 0]

df_tip_analysis_2 =  pd.merge(left=df_tip, right=df_tip_to_merge, left_on=['user_id', 'previous_order_number'], right_on=['user_id', 'order_number'], how='inner', suffixes=['', '_previous'])
df_tip_analysis_2 =  pd.merge(left=df_tip_analysis_2, right=df_tip_to_merge, left_on=['user_id', 'before_previous_order_number'], right_on=['user_id', 'order_number'], how='inner', suffixes=['', '_before_previous'])

df_tip_analysis_2 = df_tip_analysis_2[['user_id', 'order_number', 'tip', 'order_number_previous', 'tip_previous', 'order_number_before_previous', 'tip_before_previous']] 

In [ ]:
path=['tip_before_previous', 'tip_previous', 'tip']
px.parallel_categories(df_tip_analysis_2, dimensions=path, color='tip', title='Tip if tipped befored')

In [ ]:
print(f'Haben Trinkgeld bei der vor-vorherigen Bestellung gegeben: {round(df_tip_analysis_2['tip_before_previous'].mean()*100, 3)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vor-vorherigen Bestellung Trinkgeld gegeben haben:\
 {round(df_tip_analysis_2['tip'][df_tip_analysis_2['tip_before_previous']==1].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vor-vorherigen Bestellung kein Trinkgeld gegeben haben:\
 {round(df_tip_analysis_2['tip'][df_tip_analysis_2['tip_before_previous']==0].mean()*100, 1)}%')


In [ ]:
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei den beiden voherigen Bestellungen Trinkgeld gegeben haben:\
 {round(df_tip_analysis_2['tip'][(df_tip_analysis_2['tip_before_previous']==1) & (df_tip_analysis_2['tip_previous']==1)].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei den beiden voherigen Bestellungen kein Trinkgeld gegeben haben:\
 {round(df_tip_analysis_2['tip'][(df_tip_analysis_2['tip_before_previous']==0) & (df_tip_analysis_2['tip_previous']==0)].mean()*100, 1)}%')

In [ ]:
#aus 1.1 mit reduziertem Datensatz
print(f'Haben Trinkgeld bei der vorherigen Bestellung gegeben: {round(df_tip_analysis_2['tip_previous'].mean()*100, 3)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vorherigen Bestellung Trinkgeld gegeben haben:\
 {round(df_tip_analysis_2['tip'][df_tip_analysis_2['tip_previous']==1].mean()*100, 1)}%')
print(f'Haben Trinkgeld bei der nächsten Bestellung gegeben, wenn sie bei der vorherigen Bestellung kein Trinkgeld gegeben haben:\
 {round(df_tip_analysis_2['tip'][df_tip_analysis_2['tip_previous']==0].mean()*100, 1)}%')

In [ ]:
df_tip_analysis_2